In [1]:
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np
from pyspark.sql.functions import col
from pyspark.sql.functions import lit

In [3]:
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import *

In [4]:
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *

In [5]:
from pyspark.ml import Pipeline

In [6]:
dataset =spark.read.format('json').options(header=True, inferSchema=True).load("dataset_final.json")

In [7]:
ngram=NGram(n=2, inputCol="text_cleaned", outputCol="text_ngram")
hashingtf=HashingTF(inputCol="text_ngram", outputCol="text_hashing")
idf = IDF(minDocFreq = 3, inputCol="text_hashing", outputCol="text_idf")
lr = LogisticRegression(featuresCol="text_idf", labelCol="label")
train,test = dataset.randomSplit([0.8,0.2])

In [8]:
pipeline = Pipeline(stages=[ngram,hashingtf,idf,lr])

In [9]:
model = pipeline.fit(train)

In [10]:
model_result=model.transform(test)

In [11]:
evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol="label", metricName="areaUnderPR")

In [12]:
evaluator.evaluate(model_result)

0.9718963644904461

In [26]:
type(model)

pyspark.ml.pipeline.PipelineModel

In [29]:
model.save("logisticregressionmodel")